In [1]:
import serial.tools.list_ports
import re
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import psycopg2
import time

In [3]:
import psycopg2
import time

datasource = 'postgres'  
username = 'postgres'  
password = '1234'      
host = 'localhost'     
port = '5432'

In [4]:
try:
    conn = psycopg2.connect(
        dbname=datasource, 
        user=username, 
        password=password, 
        host=host, 
        port=port
    )
    print("Database connection successful")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit()
cursor = conn.cursor()
insert_query = """
                    INSERT INTO Readings (temperature1, temperature2, temperature3, temperature4, 
                            temperature5, temperature6, temperature7, temperature8, currentvalue)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
                    """ 

Database connection successful


In [ ]:
port = "COM3"
baudrate = 115200  
timeout = 1     
readings = []

# Correct placeholder for PostgreSQL
insert_query = """
INSERT INTO Readings (temperature1, temperature2, temperature3, temperature4, 
                            temperature5, temperature6, temperature7, temperature8, currentvalue)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

try:
    ser = serial.Serial(port, baudrate, timeout=timeout)
    print(f"Connected to {port}")

    while True:
        if ser.in_waiting > 0:  
            data = ser.readline().decode('utf-8').strip()
            
            str_data = str(data)
            
            if str_data.startswith("Temperature Sensor"):
                # Extracting temperature sensor index and value
                parts = str_data.split(":")
                if len(parts) == 2:
                    temp_value = parts[1].strip()
                    if temp_value == "Not connected":
                        readings.append(None)  # Append None for disconnected sensor
                    else:
                        readings.append(float(temp_value))
            
            elif str_data.startswith("Current:"):
                # Extracting current sensor value
                new_string_C = str_data.replace("Current: ", "")
                new_float_C = float(new_string_C)
                readings.append(new_float_C)
            
            if len(readings) == 9:  # Ensure all 8 temperatures + 1 current reading are present
                readings_tuple = [tuple(readings)]
                print(readings_tuple)
                
                # Insert readings into database
                cursor.executemany(insert_query, readings_tuple)
                
                # Check and delete older rows if necessary
                cursor.execute("SELECT COUNT(*) FROM Readings;")
                row_count = cursor.fetchone()[0]
                
                if row_count > 100:
                    delete_query = """
                        DELETE FROM Readings
                        WHERE id IN (
                            SELECT id
                            FROM Readings
                            ORDER BY id ASC
                            LIMIT %s
                        );
                        """
                    cursor.execute(delete_query, (row_count - 20,))
                
                conn.commit()  # Commit the changes
                readings.clear()  # Reset readings for the next batch
            
except serial.SerialException as e:
    print(f"Error: {e}")

except KeyboardInterrupt:
    print("\nExiting program.")

finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()
        print(f"Closed connection to {port}")


Connected to COM3
[(30.78, 36.91, 34.09, 33.93, 34.81, 34.49, 34.81, 34.81, -1.358)]
[(34.81, 36.26, 33.93, 35.14, 34.73, 34.81, 34.57, 34.65, -1.471)]
[(34.65, 34.25, 34.89, 34.81, 34.89, 34.81, 34.57, 35.86, -1.455)]
[(34.33, 34.81, 28.12, 34.73, 34.81, 34.73, 34.09, 34.97, -1.438)]
[(34.81, 34.09, 34.81, 34.73, 35.22, 34.81, 34.65, 34.81, -1.398)]
[(35.3, 33.77, 34.65, 34.41, 34.81, 34.89, 32.23, 35.05, -1.406)]
[(35.22, 36.1, 35.62, 35.3, 32.32, 35.38, 34.65, 35.38, -1.414)]
[(33.93, 31.27, 35.86, 34.81, 35.22, 36.1, 36.18, 36.59, -1.366)]
[(37.88, 38.84, 36.67, 35.38, 34.89, 34.81, 34.65, 34.81, -1.39)]
[(35.05, 34.81, 35.54, 36.1, 35.3, 32.8, 31.83, 32.23, -1.6)]
[(34.57, 33.93, 34.57, 34.73, 34.57, 34.81, 33.36, 35.86, -1.422)]
[(35.54, 39.57, 36.59, 36.1, 35.86, 35.22, 36.1, 34.89, -1.39)]
[(33.68, 35.38, 33.77, 35.05, 34.81, 33.44, 39.97, 35.22, -1.463)]
[(34.81, 35.7, 35.05, 34.81, 34.81, 34.09, 31.11, 35.3, -1.406)]
[(34.81, 34.49, 33.52, 35.86, 34.81, 35.3, 35.7, 37.39, -1.

In [13]:
# clear the table
table_name = 'readings'
clear_query = f"DELETE FROM {table_name};"
cursor.execute(clear_query)
conn.commit()